In [ ]:
# !pip install -U keras
!pip install git+https://github.com/JihongJu/keras-resnet3d.git
!pip install jiwer

  Cloning https://github.com/JihongJu/keras-resnet3d.git to /tmp/pip-req-build-357vc0pi
  Running command git clone -q https://github.com/JihongJu/keras-resnet3d.git /tmp/pip-req-build-357vc0pi
  Running command git submodule update --init --recursive -q
  Created wheel for keras-resnet3d: filename=keras_resnet3d-0.0.1-cp36-none-any.whl size=4773 sha256=2d933b275083700614b9c60d7492aecc53593e1a834072b7bd02965897e55c93
  Stored in directory: /tmp/pip-ephem-wheel-cache-sk50myak/wheels/98/fa/00/0dfe23f54c8f40a14c53f355effb4b198f7727d6d7f05be8b1
Successfully built keras-resnet3d


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import keras
from keras.layers import Dense,Conv2D,LSTM,Bidirectional,TimeDistributed,Input,RepeatVector
from keras.models import Model,load_model
from keras.optimizers import RMSprop

Using TensorFlow backend.


#Preprocessing

In [ ]:
import os
from math import ceil
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

In [ ]:
annonationsPath = '/content/drive/My Drive/train.xlsx'
parentFolder = '/content/drive/My Drive/Train/'
numVideos = 30
finalOutput = {}
max_length_video_id=None
max_length_video=None
X_train = []
y_train = []

In [ ]:
# import re
s= ["<S> hey, This is Me!!! <E>", 
    "<S> i am running and i like to run and <E>",
    "<S> i also like playing football. <E>"]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(s)
a = tokenizer.texts_to_sequences(s)
for i in a:
    print(f"{i} --> {tokenizer.sequences_to_texts([i])}")

In [ ]:
def textPreprocessing(data):
    #Removing Punctuations
    global max_length_video_id
    data = [s.translate(str.maketrans('', '', string.punctuation)) for s in data]

    #Converting text to lower case and calculating max length
    size_1 = []
    for i in range(len(data)):
        data[i] = data[i].lower()
        size_1.append(len(data[i].split()))

    #Max Sentence Length
    max_sentence_length = max(size_1)

    #Converting Text into Tokens
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)
    seq = tokenizer.texts_to_sequences(data)
    seq = pad_sequences(seq, maxlen=max_sentence_length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    # print(max_sentence_length,vocab_size)

    return seq,max_sentence_length,vocab_size

def preprocess_sentences():
    global max_length_video
    annonationData = pd.read_excel(annonationsPath)
    seq,max_length,vocab_size = textPreprocessing(annonationData.iloc[:numVideos,3].values)
    id = annonationData.iloc[:numVideos,0].values
    for i in range(len(id)):
        finalOutput[id[i]] = seq[i]
    return max_length,vocab_size

def framing(folderImages,window_size=8,stride=4):
    num_images = len(folderImages)
    meta_frames = int(np.floor((num_images-window_size)/stride)) + 1
    output = []
    for i in range(meta_frames):
        imgs=[]
        for j in range(i*stride,window_size+i*stride):
            imgs.append(folderImages[j])
        output.append(imgs)
    return output

def getMaxFrames():
    count=0
    for folder in os.listdir(parentFolder)[:numVideos]:
        for subfolder in os.listdir(parentFolder+folder):
            childPath = parentFolder+folder+'/'+subfolder
            numFrames = len(os.listdir(childPath))
            if(numFrames>count):
                count=numFrames
    return count

def preprocess_videos():
    max_num_frames = getMaxFrames()
    # print('Max Frames',max_num_frames)
    for folder in tqdm(os.listdir(parentFolder)[:numVideos]):
        # print(folder)
        for subfolder in os.listdir(parentFolder+folder):
            childPath = parentFolder+folder+'/'+subfolder
            video = []
            # print(len(os.listdir(childPath)))
            numBlankFrames = max_num_frames - len(os.listdir(childPath))
            for image in os.listdir(childPath):
                img = cv2.imread(childPath+'/'+image)
                img = cv2.resize(img,(224,224), interpolation = cv2.INTER_AREA)
                img = img/255.0
                img = np.array(img,dtype='float16')
                video.append(img)
            blankFrame = np.zeros((224,224,3),np.uint8)
            for i in range(numBlankFrames):
                video.append(blankFrame)
            # print(len(video))
            X_train.append(framing(video))
            y_train.append(finalOutput[folder])


In [ ]:
max_length,vocab_size = preprocess_sentences()
preprocess_videos()
X_train = np.array(X_train)
print(X_train.shape)
y_train = np.array(y_train)
print(y_train.shape)

np.save('/content/drive/My Drive/X_train.npy',X_train)
np.save('/content/drive/My Drive/y_train.npy',y_train)
print(max_length,vocab_size)

In [ ]:
import json
with open('/content/drive/My Drive/meta.json','w') as file:
  json.dump({'max_length':max_length,'vocab_size':vocab_size},file)

In [ ]:
#Importing the Data 
X_train = np.load('/content/drive/My Drive/X_train.npy')
y_train = np.load('/content/drive/My Drive/y_train.npy')
print(X_train.shape)
print(y_train.shape)

(30, 53, 8, 224, 224, 3)
(30, 23)


In [ ]:
from sklearn.utils import shuffle
X_train,y_train = shuffle(X_train,y_train)

In [ ]:
import json
with open('/content/drive/My Drive/meta.json') as f:
  output= json.load(f)
max_length = output['max_length']
vocab_size = output['vocab_size']

In [ ]:
input_shape = (30,53,8,224,224,3)
out_timesteps = 23
out_vocab = 157

In [ ]:
# Defining the resnet3d model
from resnet3d import Resnet3DBuilder
resnet3dmodel = Resnet3DBuilder.build_resnet_18((input_shape[2], input_shape[3], input_shape[4], input_shape[5]), 0)
# layer_names = [layer.name for layer in model.layers]
last_layer = resnet3dmodel.layers[-2]
base_model = Model(inputs = resnet3dmodel.input, outputs=last_layer.output)

In [ ]:
base_model_1 = load_model('/content/drive/My Drive/model_normalized.h5')
last_layer = base_model_1.layers[-2]
base_model = Model(inputs=base_model_1.input,outputs=last_layer.output)
base_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 224, 224 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 15, 112, 112, 65920       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 15, 112, 112, 256         conv3d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 15, 112, 112, 0           batch_normalization_1[0][0]      
______________________________________________________________________________________________

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

base_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 224, 224 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 15, 112, 112, 65920       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 15, 112, 112, 256         conv3d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 15, 112, 112, 0           batch_normalization_1[0][0]      
______________________________________________________________________________________________

In [ ]:
max_length

23

In [ ]:
# Defining the main model

#Input layer
input_tensor = Input(shape=(input_shape[1],input_shape[2],input_shape[3],input_shape[4],input_shape[5]))
print(input_tensor)
#Feature Extractor
resnet = TimeDistributed(base_model)(input_tensor)
print(resnet)
#Encoder
bidirectional_lstm_1 = Bidirectional(LSTM(1024,return_sequences=True))(resnet)
print(bidirectional_lstm_1)
bidirectional_lstm_2 = Bidirectional(LSTM(1024))(bidirectional_lstm_1)
print(bidirectional_lstm_2)

#Decoder
repeatVector = RepeatVector(out_timesteps)(bidirectional_lstm_2)
lstm_1 = LSTM(1024, return_sequences=True)(repeatVector)
print(lstm_1)
#Output Dense layer
dense = Dense(256,activation='relu')(lstm_1)
output_tensor = Dense(out_vocab, activation='softmax')(dense)
print(output_tensor)

model = Model(inputs=input_tensor,outputs=output_tensor)

Tensor("input_2:0", shape=(None, 53, 8, 224, 224, 3), dtype=float32)
Tensor("time_distributed_1/Reshape_1:0", shape=(None, 53, 512), dtype=float32)
Tensor("bidirectional_1/concat:0", shape=(None, 53, 2048), dtype=float32)
Tensor("bidirectional_2/concat:0", shape=(None, 2048), dtype=float32)
Tensor("lstm_3/transpose_1:0", shape=(None, 23, 1024), dtype=float32)
Tensor("dense_3/truediv:0", shape=(None, 23, 157), dtype=float32)


In [ ]:
#Model compilation
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 53, 8, 224, 224, 3 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 53, 512)           33214720  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 53, 2048)          12591104  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 2048)              25174016  
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 23, 2048)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 23, 1024)          12587008  
_________________________________________________________________
dense_2 (Dense)              (None, 23, 256)           2624

In [ ]:
# filename = 'model.h1.6_april_20'
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# train model
history = model.fit(X_train, y_train.reshape(y_train.shape[0], y_train.shape[1], 1),
                    epochs=20, batch_size=2, validation_split = 0.2, 
                    verbose=1,shuffle=True)